In [11]:
using Pkg; Pkg.instantiate(); Pkg.activate("..")
using Plots
using CSV
using DataFrames
using StatsPlots
using PlotlyJS
using Interact

 Activating environment at `C:\Users\Sharan Yalburgi\Documents\code\my-binders\Project.toml`
  Resolving package versions...
Updating `C:\Users\Sharan Yalburgi\Documents\code\my-binders\Project.toml`
  [c601a237] + Interact v0.10.3
Updating `C:\Users\Sharan Yalburgi\Documents\code\my-binders\Manifest.toml`
  [70588ee8] + CSSUtil v0.1.1
  [c601a237] + Interact v0.10.3
  [d3863d7c] + InteractBase v0.10.6
  [bcebb21b] + Knockout v0.2.4
┌ Info: Precompiling Interact [c601a237-2ae4-5e1e-952c-7a85b0c7eef1]
└ @ Base loading.jl:1278


In [55]:
Pkg.add("WebIO")

  Resolving package versions...
Updating `C:\Users\Sharan Yalburgi\Documents\code\my-binders\Project.toml`
  [0f1e0344] + WebIO v0.8.15
No Changes to `C:\Users\Sharan Yalburgi\Documents\code\my-binders\Manifest.toml`


In [21]:
results = Dict(
    0.5=>Dict(
        "dropout"=>CSV.read(joinpath("data", "dropout_batchsize=256_epochs=100_prior_var=0.5_η=0.csv"), DataFrame),
        "vanilla"=>CSV.read(joinpath("data", "vanilla_batchsize=256_epochs=100_prior_var=0.5_η=0.csv"), DataFrame),
        "map"=>CSV.read(joinpath("data", "MAP_batchsize=256_epochs=100_mcmc_steps=1_prior_var=0.5_η=0.csv"), DataFrame),
#         0.1=>CSV.read(joinpath("data", "MALA_batchsize=256_epochs=100_initial_ϵ=0.1_mcmc_steps=1_prior_var=0.5_η=0.csv"), DataFrame),
        0.01=>CSV.read(joinpath("data", "MALA_batchsize=256_epochs=100_initial_ϵ=0.01_mcmc_steps=1_prior_var=0.5_η=0.csv"), DataFrame),
        0.001=>CSV.read(joinpath("data", "MALA_batchsize=256_epochs=100_initial_ϵ=0.001_mcmc_steps=1_prior_var=0.5_η=0.csv"), DataFrame),
        0.0001=>CSV.read(joinpath("data", "MALA_batchsize=256_epochs=100_initial_ϵ=0.0001_mcmc_steps=1_prior_var=0.5_η=0.csv"), DataFrame),
    ),
    0.05=>Dict(
        "dropout"=>CSV.read(joinpath("data", "dropout_batchsize=256_epochs=100_prior_var=0.05_η=0.csv"), DataFrame),
        "vanilla"=>CSV.read(joinpath("data", "vanilla_batchsize=256_epochs=100_prior_var=0.05_η=0.csv"), DataFrame),
        "map"=>CSV.read(joinpath("data", "MAP_batchsize=256_epochs=100_mcmc_steps=1_prior_var=0.05_η=0.csv"), DataFrame),
#         0.1=>CSV.read(joinpath("data", "MALA_batchsize=256_epochs=100_initial_ϵ=0.1_mcmc_steps=1_prior_var=0.05_η=0.csv"), DataFrame),
        0.01=>CSV.read(joinpath("data", "MALA_batchsize=256_epochs=100_initial_ϵ=0.01_mcmc_steps=1_prior_var=0.05_η=0.csv"), DataFrame),
        0.001=>CSV.read(joinpath("data", "MALA_batchsize=256_epochs=100_initial_ϵ=0.001_mcmc_steps=1_prior_var=0.05_η=0.csv"), DataFrame),
        0.0001=>CSV.read(joinpath("data", "MALA_batchsize=256_epochs=100_initial_ϵ=0.0001_mcmc_steps=1_prior_var=0.05_η=0.csv"), DataFrame),
    ),
    )

Dict{Float64,Dict{Any,DataFrame}} with 2 entries:
  0.5  => Dict{Any,DataFrame}(0.0001=>100×8 DataFrame…
  0.05 => Dict{Any,DataFrame}(0.0001=>100×8 DataFrame…

In [57]:
ϵ = widget([10.0^i for i in (-4):(-2)], label="ϵ")
σ = widget([5*10.0^i for i in (-2):(-1)], label="σ")
MAP = widget(Dict("ON" => 1, "OFF" => 0), label="Show MAP results");
vanilla = widget(Dict("ON" => 1, "OFF" => 0), label="Show vanilla results");
dropout = widget(Dict("ON" => 1, "OFF" => 0), label="Show dropout results");
function create_plot(ϵ, σ, MAP, vanilla, dropout)
    accs_plts = GenericTrace[]
    cliks_plts = GenericTrace[]
    pliks_plts = GenericTrace[]
    aps_plts = GenericTrace[]

    push!(accs_plts, PlotlyJS.scatter(;x=1:length(results[σ][ϵ].test_acc), y=results[σ][ϵ].test_acc, name="MALA test", linewidth=0.5, range=(0.95, 1.0)))
    push!(cliks_plts, PlotlyJS.scatter(;x=1:length(results[σ][ϵ].test_clik), y=results[σ][ϵ].test_clik, name="MALA test", linewidth=0.5, range=(0.95, 1.0)))
    push!(pliks_plts, PlotlyJS.scatter(;x=1:length(results[σ][ϵ].test_plik), y=results[σ][ϵ].test_plik, name="MALA test", linewidth=0.5, range=(0.95, 1.0)))
    
    push!(accs_plts, PlotlyJS.scatter(;x=1:length(results[σ][ϵ].train_acc), y=results[σ][ϵ].train_acc, name="MALA train", linewidth=0.5, range=(0.95, 1.0)))
    push!(cliks_plts, PlotlyJS.scatter(;x=1:length(results[σ][ϵ].train_clik), y=results[σ][ϵ].train_clik, name="MALA train", linewidth=0.5, range=(0.95, 1.0)))
    push!(pliks_plts, PlotlyJS.scatter(;x=1:length(results[σ][ϵ].train_plik), y=results[σ][ϵ].train_plik, name="MALA train", linewidth=0.5, range=(0.95, 1.0)))

    push!(aps_plts, PlotlyJS.scatter(;x=1:length(results[σ][ϵ].accept_prob), y=results[σ][ϵ].accept_prob, name="", linewidth=0.5, range=(0.95, 1.0)))
    
    if MAP==1
        push!(accs_plts, PlotlyJS.scatter(;x=1:length(results[σ]["map"].test_acc), y=results[σ]["map"].test_acc, name="MAP test", linewidth=0.5, range=(0.95, 1.0)))
        push!(cliks_plts, PlotlyJS.scatter(;x=1:length(results[σ]["map"].test_clik), y=results[σ]["map"].test_clik, name="MAP test", linewidth=0.5, range=(0.95, 1.0)))
        push!(pliks_plts, PlotlyJS.scatter(;x=1:length(results[σ]["map"].test_plik), y=results[σ]["map"].test_plik, name="MAP test", linewidth=0.5, range=(0.95, 1.0)))
        push!(accs_plts, PlotlyJS.scatter(;x=1:length(results[σ]["map"].train_acc), y=results[σ]["map"].train_acc, name="MAP train", linewidth=0.5, range=(0.95, 1.0)))
        push!(cliks_plts, PlotlyJS.scatter(;x=1:length(results[σ]["map"].train_clik), y=results[σ]["map"].train_clik, name="MAP train", linewidth=0.5, range=(0.95, 1.0)))
        push!(pliks_plts, PlotlyJS.scatter(;x=1:length(results[σ]["map"].train_plik), y=results[σ]["map"].train_plik, name="MAP train", linewidth=0.5, range=(0.95, 1.0)))
    end
    if vanilla==1
        push!(accs_plts, PlotlyJS.scatter(;x=1:length(results[σ]["vanilla"].test_acc), y=results[σ]["vanilla"].test_acc, name="Vanilla test", linewidth=0.5, range=(0.95, 1.0)))
        push!(cliks_plts, PlotlyJS.scatter(;x=1:length(results[σ]["vanilla"].test_clik), y=results[σ]["vanilla"].test_clik, name="Vanilla test", linewidth=0.5, range=(0.95, 1.0)))
        push!(accs_plts, PlotlyJS.scatter(;x=1:length(results[σ]["vanilla"].train_acc), y=results[σ]["vanilla"].train_acc, name="Vanilla train", linewidth=0.5, range=(0.95, 1.0)))
        push!(cliks_plts, PlotlyJS.scatter(;x=1:length(results[σ]["vanilla"].train_clik), y=results[σ]["vanilla"].train_clik, name="Vanilla train", linewidth=0.5, range=(0.95, 1.0)))
    end
    if dropout==1
        push!(accs_plts, PlotlyJS.scatter(;x=1:length(results[σ]["dropout"].test_acc), y=results[σ]["dropout"].test_acc, name="Dropout test", linewidth=0.5, range=(0.95, 1.0)))
        push!(cliks_plts, PlotlyJS.scatter(;x=1:length(results[σ]["dropout"].test_clik), y=results[σ]["dropout"].test_clik, name="Dropout test", linewidth=0.5, range=(0.95, 1.0)))
        push!(accs_plts, PlotlyJS.scatter(;x=1:length(results[σ]["dropout"].train_acc), y=results[σ]["dropout"].train_acc, name="Dropout train", linewidth=0.5, range=(0.95, 1.0)))
        push!(cliks_plts, PlotlyJS.scatter(;x=1:length(results[σ]["dropout"].train_clik), y=results[σ]["dropout"].train_clik, name="Dropout train", linewidth=0.5, range=(0.95, 1.0)))
    end
    return [PlotlyJS.Plot(accs_plts) PlotlyJS.Plot(cliks_plts) PlotlyJS.Plot(pliks_plts) PlotlyJS.Plot(aps_plts)]
#     return PlotlyJS.plot(plt)
end
interactive_plot = map(create_plot, ϵ, σ, MAP, vanilla, dropout)

vbox(
    hbox(ϵ, σ), # stack horizontally
    hbox(MAP, vanilla, dropout),
    interactive_plot
)

(div { style=Dict("display" => "flex","flex-direction" => "column") }
  (div { style=Dict("display" => "flex","flex-direction" => "row") }
    Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["ϵ"], Dict{Symbol,Any}(:className => "interact ",:style => Dict{Any,Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol,Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(:max => 3,:min => 1,:attributes => Dict{Any,Any}(:type => "range",Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient" => "horizontal"),:step => 1,:className => "slider slider is-fullwidth",:style => Dict{Any,Any}()))], Dict{Symbol,Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol,Any}(:className => "interact-flex-row-right"))], Dict{Symbol,Any}(:className => "interact-flex-row interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing),"index" => (Observable{Any} with 2 listeners. Value:
2, nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "C:\\Users\\Sharan Yalburgi\\.julia\\packages\\Knockout\\IP1uR\\src\\..\\assets\\knockout.js"), Asset("js", "knockout_punches", "C:\\Users\\Sharan Yalburgi\\.julia\\packages\\Knockout\\IP1uR\\src\\..\\assets\\knockout_punches.js"), Asset("js", nothing, "C:\\Users\\Sharan Yalburgi\\.julia\\packages\\InteractBase\\Zfu5P\\src\\..\\assets\\all.js"), Asset("css", nothing, "C:\\Users\\Sharan Yalburgi\\.julia\\packages\\InteractBase\\Zfu5P\\src\\..\\assets\\style.css"), Asset("css", nothing, "C:\\Users\\Sharan Yalburgi\\.julia\\packages\\Interact\\SbgIk\\src\\..\\assets\\bulma_confined.min.css")], Dict{Any,Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set{AbstractConnection}(), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x000000001600dfb0, Task (runnable) @0x000000001600dfb0), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"0.0001\",\"0.001\",\"0.01\"],\"changes\":

In [41]:
Interact.@manipulate for ϵ=[10.0^i for i in (-4):(-2)],
        σ=[5*10.0^i for i in (-2):(-1)],
        MAP=Dict("ON" => 1, "OFF" => 0),
        vanilla=Dict("ON" => 1, "OFF" => 0),
        dropout=Dict("ON" => 1, "OFF" => 0)
    plts = GenericTrace[]
    push!(plts, PlotlyJS.scatter(;x=1:length(results[σ][ϵ].test_acc), y=results[σ][ϵ].test_acc, name="MALA test", linewidth=0.5, ylims=(0.95, 1.0)))
    push!(plts, PlotlyJS.scatter(;x=1:length(results[σ][ϵ].train_acc), y=results[σ][ϵ].train_acc, name="MALA train", linewidth=0.5, ylims=(0.95, 1.0)))
    if MAP==1
        push!(plts, PlotlyJS.scatter(;x=1:length(results[σ]["map"].test_acc), y=results[σ]["map"].test_acc, name="MAP test", linewidth=0.5, ylims=(0.95, 1.0)))
        push!(plts, PlotlyJS.scatter(;x=1:length(results[σ]["map"].train_acc), y=results[σ]["map"].train_acc, name="MAP train", linewidth=0.5, ylims=(0.95, 1.0)))
    end
    if vanilla==1
        push!(plts, PlotlyJS.scatter(;x=1:length(results[σ]["vanilla"].test_acc), y=results[σ]["vanilla"].test_acc, name="Vanilla test", linewidth=0.5, ylims=(0.95, 1.0)))
        push!(plts, PlotlyJS.scatter(;x=1:length(results[σ]["vanilla"].train_acc), y=results[σ]["vanilla"].train_acc, name="Vanilla train", linewidth=0.5, ylims=(0.95, 1.0)))
    end
    if dropout==1
        push!(plts, PlotlyJS.scatter(;x=1:length(results[σ]["dropout"].test_acc), y=results[σ]["dropout"].test_acc, name="Dropout test", linewidth=0.5, ylims=(0.95, 1.0)))
        push!(plts, PlotlyJS.scatter(;x=1:length(results[σ]["dropout"].train_acc), y=results[σ]["dropout"].train_acc, name="Dropout train", linewidth=0.5, ylims=(0.95, 1.0)))
    end
    #return PlotlyJS.Plot(plts)
    
    PlotlyJS.plot(PlotlyJS.Plot(plts))
end


Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["ϵ"], Dict{Symbol,Any}(:className => "interact ",:style => Dict{Any,Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol,Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(:max => 3,:min => 1,:attributes => Dict{Any,Any}(:type => "range",Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient" => "horizontal"),:step => 1,:className => "slider slider is-fullwidth",:style => Dict{Any,Any}()))], Dict{Symbol,Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol,Any}(:className => "interact-flex-row-right"))], Dict{Symbol,Any}(:className => "interact-flex-row interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing),"index" => (Observable{Any} with 2 listeners. Value:
2, nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "C:\\Users\\Sharan Yalburgi\\.julia\\packages\\Knockout\\IP1uR\\src\\..\\assets\\knockout.js"), Asset("js", "knockout_punches", "C:\\Users\\Sharan Yalburgi\\.julia\\packages\\Knockout\\IP1uR\\src\\..\\assets\\knockout_punches.js"), Asset("js", nothing, "C:\\Users\\Sharan Yalburgi\\.julia\\packages\\InteractBase\\Zfu5P\\src\\..\\assets\\all.js"), Asset("css", nothing, "C:\\Users\\Sharan Yalburgi\\.julia\\packages\\InteractBase\\Zfu5P\\src\\..\\assets\\style.css"), Asset("css", nothing, "C:\\Users\\Sharan Yalburgi\\.julia\\packages\\Interact\\SbgIk\\src\\..\\assets\\bulma_confined.min.css")], Dict{Any,Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set{AbstractConnection}(), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x000000004e243c70, Task (runnable) @0x000000004e243c70), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"0.0001\",\"0.001\",\"0.01\"],\"changes\":WebIO.getval({\"name\":\"changes\",\"scope\":\"7476139111609781202\",\"id\":\"592146655197775253\",

In [68]:
ϵ = widget([10.0^i for i in (-4):(-2)], label="ϵ")
σ = widget([5*10.0^i for i in (-2):(-1)], label="σ")
MAP = widget(Dict("ON" => 1, "OFF" => 0), label="Show MAP results");
vanilla = widget(Dict("ON" => 1, "OFF" => 0), label="Show vanilla results");
dropout = widget(Dict("ON" => 1, "OFF" => 0), label="Show dropout results");
function create_plot(ϵ, σ, MAP, vanilla, dropout)
    accs_plt = Plots.plot(title="Accuracy", legendfontsize=5, legend=:bottomright, titlefontsize=10)
    cliks_plt = Plots.plot(title="Categorical Likelihood", legendfontsize=5, legend=:bottomright, titlefontsize=10)
    pliks_plt = Plots.plot(title="Posterior Density", legendfontsize=5, legend=:bottomright, titlefontsize=10)
    aps_plt = Plots.plot(title="Accept Probs", legendfontsize=5, legend=:bottomright, titlefontsize=10)

    Plots.plot!(accs_plt, 1:length(results[σ][ϵ].test_acc), results[σ][ϵ].test_acc, label="MALA test", linewidth=0.8, ylims=(0.95, 1.0))
    Plots.plot!(cliks_plt, 1:length(results[σ][ϵ].test_clik), results[σ][ϵ].test_clik, label="MALA test", linewidth=0.8, ylims=(-0.2, 0))
    Plots.plot!(pliks_plt, 1:length(results[σ][ϵ].test_plik), results[σ][ϵ].test_plik, label="MALA test", linewidth=0.8)
    Plots.plot!(accs_plt, 1:length(results[σ][ϵ].train_acc), results[σ][ϵ].train_acc, label="MALA train", linewidth=0.8, ylims=(0.95, 1.0))
    Plots.plot!(cliks_plt, 1:length(results[σ][ϵ].train_clik), results[σ][ϵ].train_clik, label="MALA train", linewidth=0.8, ylims=(-0.2, 0))
    Plots.plot!(pliks_plt, 1:length(results[σ][ϵ].train_plik), results[σ][ϵ].train_plik, label="MALA train", linewidth=0.8)

    Plots.plot!(aps_plt, 1:length(results[σ][ϵ].accept_prob), results[σ][ϵ].accept_prob, label="MALA train", linewidth=0.8)

    if MAP==1
        Plots.plot!(accs_plt, 1:length(results[σ]["map"].test_acc), results[σ]["map"].test_acc, label="MAP test", linewidth=0.8, ylims=(0.95, 1.0))
        Plots.plot!(cliks_plt, 1:length(results[σ]["map"].test_clik), results[σ]["map"].test_clik, label="MAP test", linewidth=0.8, ylims=(-0.2, 0))
        Plots.plot!(pliks_plt, 1:length(results[σ]["map"].test_plik), results[σ]["map"].test_plik, label="MAP test", linewidth=0.8)
        Plots.plot!(accs_plt, 1:length(results[σ]["map"].train_acc), results[σ]["map"].train_acc, label="MAP train", linewidth=0.8, ylims=(0.95, 1.0))
        Plots.plot!(cliks_plt, 1:length(results[σ]["map"].train_clik), results[σ]["map"].train_clik, label="MAP train", linewidth=0.8, ylims=(-0.2, 0))
        Plots.plot!(pliks_plt, 1:length(results[σ]["map"].train_plik), results[σ]["map"].train_plik, label="MAP train", linewidth=0.8)
    end
    if vanilla==1
        Plots.plot!(accs_plt, 1:length(results[σ]["vanilla"].test_acc), results[σ]["vanilla"].test_acc, label="Vanilla test", linewidth=0.8, ylims=(0.95, 1.0))
        Plots.plot!(cliks_plt, 1:length(results[σ]["vanilla"].test_clik), results[σ]["vanilla"].test_clik, label="Vanilla test", linewidth=0.8, ylims=(-0.2, 0))
        Plots.plot!(accs_plt, 1:length(results[σ]["vanilla"].train_acc), results[σ]["vanilla"].train_acc, label="Vanilla train", linewidth=0.8, ylims=(0.95, 1.0))
        Plots.plot!(cliks_plt, 1:length(results[σ]["vanilla"].train_clik), results[σ]["vanilla"].train_clik, label="Vanilla train", linewidth=0.8, ylims=(-0.2, 0))
    end
    if dropout==1
        Plots.plot!(accs_plt, 1:length(results[σ]["dropout"].test_acc), results[σ]["dropout"].test_acc, label="Dropout test", linewidth=0.8, ylims=(0.95, 1.0))
        Plots.plot!(cliks_plt, 1:length(results[σ]["dropout"].test_clik), results[σ]["dropout"].test_clik, label="Dropout test", linewidth=0.8, ylims=(-0.2, 0))
        Plots.plot!(accs_plt, 1:length(results[σ]["dropout"].train_acc), results[σ]["dropout"].train_acc, label="Dropout train", linewidth=0.8, ylims=(0.95, 1.0))
        Plots.plot!(cliks_plt, 1:length(results[σ]["dropout"].train_clik), results[σ]["dropout"].train_clik, label="Dropout train", linewidth=0.8, ylims=(-0.2, 0))
    end

    Plots.plot(accs_plt, cliks_plt, pliks_plt, aps_plt, layout=(1,4), size=(4,1).*300)
end
interactive_plot = map(create_plot, ϵ, σ, MAP, vanilla, dropout)

vbox(
    hbox(ϵ, σ), # stack horizontally
    hbox(MAP, vanilla, dropout),
    interactive_plot
)

(div { style=Dict("display" => "flex","flex-direction" => "column") }
  (div { style=Dict("display" => "flex","flex-direction" => "row") }
    Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["ϵ"], Dict{Symbol,Any}(:className => "interact ",:style => Dict{Any,Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol,Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(:max => 3,:min => 1,:attributes => Dict{Any,Any}(:type => "range",Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient" => "horizontal"),:step => 1,:className => "slider slider is-fullwidth",:style => Dict{Any,Any}()))], Dict{Symbol,Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol,Any}(:className => "interact-flex-row-right"))], Dict{Symbol,Any}(:className => "interact-flex-row interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing),"index" => (Observable{Any} with 2 listeners. Value:
2, nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "C:\\Users\\Sharan Yalburgi\\.julia\\packages\\Knockout\\IP1uR\\src\\..\\assets\\knockout.js"), Asset("js", "knockout_punches", "C:\\Users\\Sharan Yalburgi\\.julia\\packages\\Knockout\\IP1uR\\src\\..\\assets\\knockout_punches.js"), Asset("js", nothing, "C:\\Users\\Sharan Yalburgi\\.julia\\packages\\InteractBase\\Zfu5P\\src\\..\\assets\\all.js"), Asset("css", nothing, "C:\\Users\\Sharan Yalburgi\\.julia\\packages\\InteractBase\\Zfu5P\\src\\..\\assets\\style.css"), Asset("css", nothing, "C:\\Users\\Sharan Yalburgi\\.julia\\packages\\Interact\\SbgIk\\src\\..\\assets\\bulma_confined.min.css")], Dict{Any,Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set{AbstractConnection}(), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x000000004f74a850, Task (runnable) @0x000000004f74a850), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"0.0001\",\"0.001\",\"0.01\"],\"changes\":